<a href="https://colab.research.google.com/github/sahupra1357/HousingRateRegression/blob/main/cnn_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/emanhamed/Houses-dataset.git

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 26.72 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [2]:
!pip install requests

In [3]:
import requests
# Save datagenerators as file to colab working directory
# If you are using GitHub, make sure you get the "Raw" version of the code
url = 'https://raw.githubusercontent.com/sahupra1357/HousingRateRegression/main/pyimagesearch/datasets.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('datasets.py', 'w') as f:
    f.write(r.text)

url = 'https://raw.githubusercontent.com/sahupra1357/HousingRateRegression/main/pyimagesearch/models.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('models.py', 'w') as f:
    f.write(r.text)

import datasets
import models

In [4]:
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import locale
import os

In [9]:
!ls 

datasets.py  Houses-dataset  models.py	__pycache__  sample_data


In [11]:
!ls Houses-dataset/'Houses Dataset'

100_bathroom.jpg  221_bathroom.jpg  342_bathroom.jpg  463_bathroom.jpg
100_bedroom.jpg   221_bedroom.jpg   342_bedroom.jpg   463_bedroom.jpg
100_frontal.jpg   221_frontal.jpg   342_frontal.jpg   463_frontal.jpg
100_kitchen.jpg   221_kitchen.jpg   342_kitchen.jpg   463_kitchen.jpg
101_bathroom.jpg  222_bathroom.jpg  343_bathroom.jpg  464_bathroom.jpg
101_bedroom.jpg   222_bedroom.jpg   343_bedroom.jpg   464_bedroom.jpg
101_frontal.jpg   222_frontal.jpg   343_frontal.jpg   464_frontal.jpg
101_kitchen.jpg   222_kitchen.jpg   343_kitchen.jpg   464_kitchen.jpg
102_bathroom.jpg  223_bathroom.jpg  344_bathroom.jpg  465_bathroom.jpg
102_bedroom.jpg   223_bedroom.jpg   344_bedroom.jpg   465_bedroom.jpg
102_frontal.jpg   223_frontal.jpg   344_frontal.jpg   465_frontal.jpg
102_kitchen.jpg   223_kitchen.jpg   344_kitchen.jpg   465_kitchen.jpg
103_bathroom.jpg  224_bathroom.jpg  345_bathroom.jpg  466_bathroom.jpg
103_bedroom.jpg   224_bedroom.jpg   345_bedroom.jpg   466_bedroom.jpg
103_frontal.jpg 

In [14]:
!ls Houses-dataset/'Houses Dataset' | wc -l

2141


In [15]:
!cat Houses-dataset/'Houses Dataset'/HousesInfo.txt

4 4 4053 85255 869500
4 3 3343 36372 865200
3 4 3923 85266 889000
5 5 4022 85262 910000
3 4 4116 85266 971226
4 5 4581 85266 1249000
3 4 2544 85262 799000
4 5 5524 85266 1698000
3 4 4229 85255 1749000
4 5 3550 85262 1500000
5 5 4829 85266 519200
4 4 3428 85255 1039000
5 3 5462 85266 799000
4 4 4021 85266 889000
5 5 4406 85266 700000
4 4 3721 85255 500000
5 3 3710 85331 740000
3 4 2748 85255 725000
5 4 4190 85255 1199000
3 3.5 4143 85266 925000
3 3.5 4229 85255 1749000
6 6.5 5963 85262 1595000
3 3 2685 85255 799900
5 5 5677 85377 1375000
4 4.5 4031 85262 1345000
4 4 4954 85262 1290000
4 4.5 3550 85262 962500
5 4.5 4180 85266 875000
4 4 4464 85377 1650000
5 4.5 4829 85266 1199000
5 3 2520 93446 789000
4 3.5 3428 85255 1039000
3 2 1802 93446 365000
3 3.5 2873 85262 1695000
4 4 3260 85377 999000
3 4.5 3893 85255 1294000
5 5.5 3932 85262 1089000
3 3.5 2792 85255 799900
3 4.5 4228 85255 1229000
3 3 2146 93446 455000
4 1 9583 98021 395000
5 2 7627 98021 638940
3 2 2153 98021 435000
3 2.5 2014

In [16]:
# construct the path to the input .txt file that contains information
# on each house in the dataset and then load the dataset
print("[INFO] loading house attributes...")
inputPath = os.path.sep.join(["Houses-dataset/Houses Dataset", "HousesInfo.txt"])
inputPath

[INFO] loading house attributes...


'Houses-dataset/Houses Dataset/HousesInfo.txt'

In [17]:
df = datasets.load_house_attributes(inputPath)

In [18]:
# load the house images and then scale the pixel intensities to the
# range [0, 1]
print("[INFO] loading house images...")
images = datasets.load_house_images(df, "Houses-dataset/Houses Dataset")
#images = images / 255.0

[INFO] loading house images...


In [19]:
images.shape

(362, 64, 64, 3)

In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [ ]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (will lead to better
# training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice
# create our Convolutional Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# price *predictions* and the *actual prices*
model = models.create_cnn(64, 64, 3, regress=True)
opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(x=trainImagesX, y=trainY, 
    validation_data=(testImagesX, testY),
    epochs=200, batch_size=8)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[INFO] training model...
Epoch 1/200
34/34 [==============================] - 4s 65ms/step - loss: 630.9975 - val_loss: 203.6262
Epoch 2/200
34/34 [==============================] - 2s 54ms/step - loss: 427.9910 - val_loss: 316.8934
Epoch 3/200
34/34 [==============================] - 2s 55ms/step - loss: 272.3477 - val_loss: 115.6811
Epoch 4/200
34/34 [==============================] - 2s 54ms/step - loss: 257.2148 - val_loss: 630.4659
Epoch 5/200
34/34 [==============================] - 2s 54ms/step - loss: 166.0401 - val_loss: 1001.8469
Epoch 6/200
34/34 [==============================] - 2s 56ms/step - loss: 148.4325 - val_loss: 1155.2451
Epoch 7/200
34/34 [==============================] - 2s 55ms/step - loss: 152.7984 - val_loss: 1333.8124
Epoch 8/200
34/34 [==============================] - 2s 54ms/step - loss: 116.7083 - val_loss: 1142.8527
Epoch 9/200
34/34 [==============================] - 2s 56ms/step - loss: 100.7070 - val_loss: 929.9512
Epoch 10/200
34/34 [===============

In [ ]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict(testImagesX)
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] predicting house prices...
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 60.47%, std: 28.68%
